In [ ]:
# LSTM
# BIDIRECTIONAL LSTM

In [ ]:
;;]\import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Dropout,LSTM,Flatten
from keras.models import Sequential

from keras.layers import SimpleRNN

import numpy as np
import pandas as pd

In [ ]:
(X_train,y_train),(X_test,y_test)=keras.datasets.mnist.load_data()

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [ ]:
X_train=X_train/255
X_test=X_test/255

In [ ]:
len(np.unique(y_test))

10

In [ ]:
model=Sequential([
        LSTM(128,activation='relu',input_shape=(28,28),return_sequences=True),
        Dropout(0.2),

        LSTM(128,activation='relu',return_sequences=False),
        Dropout(0.2),

        Flatten(),

        Dense(32,activation='relu'),
        Dropout(0.2),

        Dense(10,activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 28, 128)           80384     
                                                                 
 dropout_3 (Dropout)         (None, 28, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 flatten_1 (Flatten)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 32)                4128      
                                                                 
 dropout_5 (Dropout)         (None, 32)               

In [ ]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='sparse_categorical_crossentropy')

model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
1875/1875 [==============================] - 89s 46ms/step - loss: 0.6244 - accuracy: 0.7929 - val_loss: 0.1427 - val_accuracy: 0.9567
Epoch 2/5
1875/1875 [==============================] - 84s 45ms/step - loss: 0.1830 - accuracy: 0.9499 - val_loss: 0.1048 - val_accuracy: 0.9702
Epoch 3/5
1875/1875 [==============================] - 85s 45ms/step - loss: 0.1154 - accuracy: 0.9690 - val_loss: 0.0760 - val_accuracy: 0.9774
Epoch 4/5
1875/1875 [==============================] - 87s 46ms/step - loss: 0.0906 - accuracy: 0.9764 - val_loss: 0.0813 - val_accuracy: 0.9766
Epoch 5/5
1875/1875 [==============================] - 86s 46ms/step - loss: 0.0714 - accuracy: 0.9814 - val_loss: 0.0473 - val_accuracy: 0.9867


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dropout,Flatten,LSTM,Bidirectional,Embedding,Dense
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/NLP Practice/train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
df.label.value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [ ]:
df[df.title.isna()==True]

,id,title,author,text,label
53,53,NaN,Dairy✓ᵀᴿᵁᴹᴾ,Sounds like he has our president pegged. What ...,1
120,120,NaN,Anonymous,"Same people all the time , i dont know how you...",1
124,124,NaN,SeekSearchDestory,"You know, outside of any morality arguments, i...",1
140,140,NaN,Anonymous,There is a lot more than meets the eye to this...,1
196,196,NaN,Raffie,They got the heater turned up on high.,1
...,...,...,...,...,...
20568,20568,NaN,Cathy Milne,"Amusing comment Gary! “Those week!” So, are ...",1
20627,20627,NaN,Ramona,"No she doesn’t have more money than God, every...",1
20636,20636,NaN,Dave Lowery,Trump all the way!,1
20771,20771,NaN,Letsbereal,DYN's Statement on Last Week's Botnet Attack h...,1


In [ ]:
df=df.dropna()

In [ ]:
X=df.drop('label',axis=1)
Y=df.label
X.shape,Y.shape

((18285, 4), (18285,))

In [ ]:
m=X.copy()
m.reset_index(inplace=True)

In [ ]:

stop=stopwords.words('english')
ps=PorterStemmer()
corpus=[]

for i in range(0,len(m)):
  tokens=re.sub('[^a-zA-z]',' ',m['title'][i])
  tokens.lower()
  tokens=WordPunctTokenizer().tokenize(tokens)
  #print(tokens)
  st=[ps.stem(j) for j in tokens if not j in stop]
  st=' '.join(st)
  corpus.append(st)

In [ ]:
corpus[0]

'hous dem aid we didn even see comey letter until jason chaffetz tweet it'

In [ ]:
vocab_size=5000

oh=[one_hot(j,vocab_size) for j in corpus]
print(len(oh[0]))

14


In [ ]:
maxlen=20
pad=pad_sequences(oh,maxlen,padding='post')
pad[0]

array([2357, 2153,  953, 3940,  560, 4320, 1530, 1802,  277, 2636, 2029,
       2241, 4026,  501,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
embed=Embedding(vocab_size,40,input_length=maxlen)

In [ ]:
callback=keras.callbacks.TensorBoard(log_dir='log/',histogram_freq=1)

patience=keras.callbacks.EarlyStopping(monitor='accuracy',patience=5,restore_best_weights=True)

model=Sequential([
    embed,
    LSTM(128),
    Dense(1,'sigmoid')
    ])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 128)               86528     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 286657 (1.09 MB)
Trainable params: 286657 (1.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
print(embed.get_output_shape_at(node_index=0))
#print(LSTM.get_input_shape_at(self='LSTM',node_index=0))

(None, 20, 40)


In [ ]:
len(pad),Y.shape

(18285, (18285,))

In [ ]:
X_final=np.array(pad)
y_final=np.array(Y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42,stratify=y_final)

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((12250, 20), (12250,), (6035, 20), (6035,))

In [ ]:
model.compile('adam','binary_crossentropy','accuracy')

model.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test),callbacks=[callback,patience],batch_size=64)


Epoch 1/10
192/192 [==============================] - 19s 78ms/step - loss: 0.2978 - accuracy: 0.8568 - val_loss: 0.1779 - val_accuracy: 0.9281
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1321 - accuracy: 0.9500 - val_loss: 0.2040 - val_accuracy: 0.9249
Epoch 3/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0862 - accuracy: 0.9701 - val_loss: 0.2461 - val_accuracy: 0.9225
Epoch 4/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0588 - accuracy: 0.9808 - val_loss: 0.2932 - val_accuracy: 0.9205
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0427 - accuracy: 0.9860 - val_loss: 0.2919 - val_accuracy: 0.9205
Epoch 6/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0329 - accuracy: 0.9890 - val_loss: 0.3488 - val_accuracy: 0.9150
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0232 - accuracy: 0.9927 - val_loss: 0.3737 - val_accuracy: 0.918

In [ ]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.5, 1,0)

189/189 [==============================] - 1s 2ms/step


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3192,  228],
       [ 285, 2330]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.93      0.93      3420
           1       0.91      0.89      0.90      2615

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [ ]:

model1=Sequential([
    embed,
    Bidirectional(LSTM(128)),
    Dropout(0.3),
    Dense(1,'sigmoid')
    ])

model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 256)               173056    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 373313 (1.42 MB)
Trainable params: 373313 (1.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model1.compile('adam','binary_crossentropy','accuracy')

model1.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test),callbacks=[callback,patience],batch_size=64)


Epoch 1/10
192/192 [==============================] - 16s 52ms/step - loss: 0.1058 - accuracy: 0.9707 - val_loss: 0.2552 - val_accuracy: 0.9218
Epoch 2/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0206 - accuracy: 0.9936 - val_loss: 0.3623 - val_accuracy: 0.9218
Epoch 3/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0112 - accuracy: 0.9967 - val_loss: 0.3918 - val_accuracy: 0.9138
Epoch 4/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0103 - accuracy: 0.9975 - val_loss: 0.4064 - val_accuracy: 0.9122
Epoch 5/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0084 - accuracy: 0.9984 - val_loss: 0.4664 - val_accuracy: 0.9205
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0057 - accuracy: 0.9989 - val_loss: 0.4618 - val_accuracy: 0.9231
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0075 - accuracy: 0.9981 - val_loss: 0.4367 - val_accuracy: 0.915

In [ ]:
y_pred=model1.predict(X_test)
y_pred=np.where(y_pred > 0.5, 1,0)

189/189 [==============================] - 1s 3ms/step


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3147,  273],
       [ 222, 2393]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      3420
           1       0.90      0.92      0.91      2615

    accuracy                           0.92      6035
   macro avg       0.92      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035

